In [1]:
import re


REINDEER_TEMPLATE = (
    "^(\S*) can fly (\d*) km\/s for (\d*) seconds, but then must rest for (\d*) seconds"
)
RACE_DURATION = 2503


class Reindeer:
    
    def __init__(self, name: str, speed: int, fly_time: int, rest_time: int):
        self.name = name
        self.speed = speed
        self.fly_time = fly_time
        self.rest_time = rest_time
        
        self._full_cycle_distance = self.speed * self.fly_time
        
    @classmethod
    def from_input(cls, input_line):
        reindeer_search = re.search(REINDEER_TEMPLATE, input_line)
        return cls(
            reindeer_search.group(1), 
            int(reindeer_search.group(2)), 
            int(reindeer_search.group(3)),
            int(reindeer_search.group(4)),
        )
    
    def fly(self, duration: int) -> int:
        """Calculate how far the reindeer has traveled after `duration` seconds."""
        full_cycles = duration // (self.fly_time + self.rest_time)
        distance = full_cycles * self._full_cycle_distance
        
        remaining_duration = duration - full_cycles * (self.fly_time + self.rest_time)
        next_fly_duration = min(self.fly_time, remaining_duration)
        distance += next_fly_duration * self.speed
        
        return distance 

# Part 1

In [2]:
def test_cases():
    comet = Reindeer.from_input(
        "Comet can fly 14 km/s for 10 seconds, but then must rest for 127 seconds."
    )
    dancer = Reindeer.from_input(
        "Dancer can fly 16 km/s for 11 seconds, but then must rest for 162 seconds."
    )
    
    assert comet.fly(1) == 14
    assert comet.fly(10) == 140
    assert comet.fly(11) == 140
    assert comet.fly(12) == 140
    assert comet.fly(1000) == 1120
    
    assert dancer.fly(1) == 16
    assert dancer.fly(10) == 160
    assert dancer.fly(11) == 176
    assert dancer.fly(12) == 176
    assert dancer.fly(1000) == 1056
    
    print("Reindeer test cases passed!")

test_cases()

Reindeer test cases passed!


In [3]:
input_filename = "input.txt"

with open(input_filename) as input_file:
    reindeers = [Reindeer.from_input(line) for line in input_file.readlines()]

max_distance = 0
for reindeer in reindeers:
    max_distance = max(max_distance, reindeer.fly(RACE_DURATION))
max_distance

2655

# Part 2
In which I am lazy.

In [4]:
import collections
from typing import List, Dict


def compete(reindeers: List[Reindeer], competition_time: int) -> Dict[str, int]:
    """
    Reindeer compete using Santa's new scoring system.
    Returns their final scores.
    """
    scores = collections.defaultdict(int)  # reindeer name to their score

    for t in range(1, competition_time+1):
        curr_ranking = []  # unsorted at first
        for reindeer in reindeers:
            reindeer.fly(t)
            curr_ranking.append((reindeer.fly(t), reindeer.name))
        curr_ranking.sort(reverse=True)

        best_distance = curr_ranking[0][0]
        for distance, reindeer_name in curr_ranking:
            if distance == best_distance:
                scores[reindeer_name] += 1
                continue
            break
            
    return scores

In [5]:
def test_competition():
    comet = Reindeer.from_input(
        "Comet can fly 14 km/s for 10 seconds, but then must rest for 127 seconds."
    )
    dancer = Reindeer.from_input(
        "Dancer can fly 16 km/s for 11 seconds, but then must rest for 162 seconds."
    )
    
    assert compete([comet, dancer], 1000) == {
        "Dancer": 689,
        "Comet": 312,
    }
    print("Test competition succeeded!")
    
test_competition()

Test competition succeeded!


In [6]:
max(compete(reindeers, RACE_DURATION).values())

1059